## Silver zone transformations ##

**Adding parameter cell**

In [16]:
pOrderDate='2024-01-01'

StatementMeta(, , , Waiting, )

## **Using parameterized read**

In [2]:
# dfOrderBrz=spark.table('SalesOrderHeader')
dfOrderBrz=spark.sql(f"Select * from SalesOrderHeader where OrderDate>'{pOrderDate}'")
display(dfOrderBrz)

StatementMeta(, , , Waiting, )

SynapseWidget(Synapse.DataFrame, 29d82310-45ef-42c3-85e4-67d9b3fe0f2e)

In [2]:
dfOrderBrz.printSchema()

StatementMeta(, , , Waiting, )

root
 |-- SalesOrderID: string (nullable = true)
 |-- CustomerID: string (nullable = true)
 |-- OrderDate: string (nullable = true)
 |-- DueDate: string (nullable = true)
 |-- TotalAmount: string (nullable = true)
 |-- SourceFilename: string (nullable = true)
 |-- InsertedDateTime: timestamp (nullable = true)



Apply data type corrections using **withColumn** method and cast functions.

In [3]:
from  pyspark.sql.functions import *
dfOrderSlv=dfOrderBrz.withColumn('SalesOrderID',col('SalesOrderID').cast('int'))\
.withColumn('CustomerID',col('CustomerID').cast('int'))\
.withColumn('OrderDate',col('OrderDate').cast('timestamp'))\
.withColumn('DueDate',col('OrderDate').cast('timestamp'))\
.withColumn('TotalAmount',col('TotalAmount').cast('float'))\

dfOrderSlv.printSchema()

StatementMeta(, , , Waiting, )

root
 |-- SalesOrderID: integer (nullable = true)
 |-- CustomerID: integer (nullable = true)
 |-- OrderDate: timestamp (nullable = true)
 |-- DueDate: timestamp (nullable = true)
 |-- TotalAmount: float (nullable = true)
 |-- SourceFilename: string (nullable = true)
 |-- InsertedDateTime: timestamp (nullable = true)



Reading & applying data type conversions to Customer table

In [4]:
dfCustomerSlv=spark.table('Customer')\
.withColumn('CustomerID',col('CustomerID').cast('int'))
display(dfCustomerSlv)

StatementMeta(, , , Waiting, )

SynapseWidget(Synapse.DataFrame, 3807b96f-66dc-4b68-86fd-0c061d427d47)

Ordering rows

In [5]:
dfOrderSlv=dfOrderSlv.orderBy('SalesOrderID',desc('InsertedDateTime'))
display(dfOrderSlv)

StatementMeta(, , , Waiting, )

SynapseWidget(Synapse.DataFrame, 0ad7d809-bf20-4d80-b0be-7bcd3b842ebb)

**Identifying and storing data type conversion conflicts**

In [6]:
dfOrderErrors=dfOrderSlv.where(col('SalesOrderID').isNull()).withColumn('ErrorMessage',lit('SalesOrderID data type is icorrect'))
display(dfOrderErrors)

StatementMeta(, , , Waiting, )

SynapseWidget(Synapse.DataFrame, 09da200f-8780-406f-8a23-c5ff025d8fb7)

In [7]:
dfOrderErrors.write.format('delta').mode('append').saveAsTable('DataTypeErrors')

StatementMeta(, , , Waiting, )

Filtering out problematic rows

In [8]:
dfOrderSlv=dfOrderSlv.where(col('SalesOrderID').isNotNull())
display(dfOrderSlv)

StatementMeta(, , , Waiting, )

SynapseWidget(Synapse.DataFrame, 71ef31d6-68c2-47df-bf2f-24c47385fa6d)

**Filtering out older row versions-** using **_row_number_** window function 

In [9]:
from pyspark.sql.window import Window
window = Window.partitionBy('SalesOrderID').orderBy(desc('InsertedDateTime'))
dfOrderSlv=dfOrderSlv.withColumn('RowVersion',row_number().over(window))
display(dfOrderSlv)

StatementMeta(, , , Waiting, )

SynapseWidget(Synapse.DataFrame, f08aab10-e198-41cf-9611-d70839bc7289)

In [10]:
dfOrderSlv=dfOrderSlv.filter('RowVersion==1').orderBy('SalesOrderID')
display(dfOrderSlv)

StatementMeta(, , , Waiting, )

SynapseWidget(Synapse.DataFrame, 7b508abd-63fb-4932-b9d2-4c83e6d7ae37)

Deduplication for Customer table

In [11]:
dfCustomerSlv=dfCustomerSlv.distinct()
display(dfCustomerSlv)

StatementMeta(, , , Waiting, )

SynapseWidget(Synapse.DataFrame, 1cbaf0ba-b20c-4661-888a-0d45df2b6405)

**Extracting source folder and filenames**

In [12]:
from pyspark.sql.functions import *
dfOrderSlv=dfOrderSlv.withColumn('SourceFileName',regexp_extract(col('SourceFileName'),'(\/CSV\/.*.csv)',1))
display(dfOrderSlv)

StatementMeta(, , , Waiting, )

SynapseWidget(Synapse.DataFrame, 34e78068-89f4-4068-94f9-b476d645f821)

**Generating datetime dimension**

In [13]:
dfDate=dfOrderSlv.dropDuplicates(subset=['OrderDate'])\
    .withColumnRenamed('OrderDate','Date')\
    .withColumn('DateKey',regexp_replace(substring(col('Date'),1,10),'-','').cast('bigint'))\
    .withColumn('Month',month(col('Date')))\
    .withColumn('Day',dayofmonth(col('Date')))\
    .withColumn('Year',year(col('Date')))\
    .select('DateKey','Date','Day','Month','Year')
display(dfDate)    

StatementMeta(, , , Waiting, )

SynapseWidget(Synapse.DataFrame, ba0bd90c-ca1d-4255-8133-a55bb7f8a9ed)

**Generate surrogate keys** by using monotonically_increasing_id function

In [21]:
dfCustomerSlv=dfCustomerSlv.withColumn('CustomerKey',monotonically_increasing_id())
display(dfCustomerSlv)

StatementMeta(, , , Waiting, )

SynapseWidget(Synapse.DataFrame, ec10cb82-ba6e-4891-8410-2333fdaaf805)

Get partition count

In [19]:
dfCustomerSlv.rdd.getNumPartitions()

StatementMeta(, , , Waiting, )

1

Increasing partition count

In [16]:
dfCustomerSlv=dfCustomerSlv.repartition(4)

StatementMeta(, , , Waiting, )

Using _Coalesce_ function to reduce parallelism

In [20]:
dfCustomerSlv=dfCustomerSlv.coalesce(1)
dfCustomerSlv.rdd.getNumPartitions()

StatementMeta(, , , Waiting, )

1

In [206]:
dfCustomerSlv=dfCustomerSlv.withColumn('CustomerKey',monotonically_increasing_id())
display(dfCustomerSlv)

StatementMeta(, , , Waiting, )

SynapseWidget(Synapse.DataFrame, 7471558c-f4a9-414b-9439-001bf02ad57f)

**Joining tables to get Foreign keys**

In [207]:
dfOrderSlv=dfOrderSlv.join(dfCustomerSlv,"CustomerID")\
.join(dfDate,dfOrderSlv.OrderDate==dfDate.Date)\
.select('SalesOrderID','DateKey','CustomerKey','TotalAmount','InsertedDateTime','SourceFileName')
display(dfOrderSlv)

StatementMeta(, , , Waiting, )

SynapseWidget(Synapse.DataFrame, 1cda8c19-0c65-4b66-9f8f-bebd372a9a89)

**Destination writes**

Overwriting small table

In [208]:
dfOrderSlv.write.format('delta').mode('overwrite').saveAsTable('SlvSalesOrderHeader')
dfCustomerSlv.write.format('delta').mode('overwrite').saveAsTable('SlvCustomer')

StatementMeta(, , , Waiting, )

Incremental write for large tables

Staep 1: Create temp view

In [22]:
dfOrderSlv.createOrReplaceTempView('VwOrdersSilver')

StatementMeta(, , , Waiting, )

In [210]:
%%sql
Select * from VwOrdersSilver order by SalesOrderID

StatementMeta(, , , Waiting, )

<Spark SQL result set with 29 rows and 6 fields>

Step 2: Using Merge command

In [211]:
spark.sql('MERGE INTO SlvSalesOrderHeader T USING VwOrdersSilver S \
ON T.SalesOrderID=S.SalesOrderID \
WHEN MATCHED THEN UPDATE SET * \
WHEN NOT MATCHED THEN INSERT *')

StatementMeta(, , , Waiting, )

DataFrame[num_affected_rows: bigint, num_updated_rows: bigint, num_deleted_rows: bigint, num_inserted_rows: bigint]

**Calculating Aggregations**

In [215]:
dfOrderGld=spark.table('SlvSalesOrderHeader')\
.groupBy('CustomerKey')\
.agg(sum('TotalAmount').alias('AllTotals'),avg('TotalAmount').alias('AvgSales'))
display(dfOrderGld)

StatementMeta(, , , Waiting, )

SynapseWidget(Synapse.DataFrame, dec0f67a-ad46-4672-bd01-528486c28368)

In [217]:
dfOrderGld.write.format('delta').mode('overwrite').saveAsTable('GldSales')

StatementMeta(, , , Waiting, )

**Creating aggregate views**

In [223]:
spark.sql('CREATE OR REPLACE VIEW VwAggSales AS SELECT CustomerKey,SUM(TotalAmount) As AllSales FROM SlvSalesOrderHeader GROUP BY CustomerKey')

StatementMeta(, , , Waiting, )

DataFrame[]

In [224]:
%%sql
select * from VwAggSales

StatementMeta(, , , Waiting, )

<Spark SQL result set with 29 rows and 2 fields>